# Fase 1: Extracción de Datos de API de MoviesDataset

In [1]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select


LINK API: https://rapidapi.com/SAdrian/api/moviesdatabase

- Películas y cortos desde el 2010 hasta la actualidad. ✅

- Películas que pertenezcan a los siguientes géneros: Drama, Comedy, Action. ✅

- NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. ✅

In [29]:
#creamos el diccionario donde alamcenaremos la información
dic_API_Pelis = {'Tipo':[],
                 'Nombre':[],
                 'Año_estreno':[],
                 'Mes_estreno': [],
                 'Id':[]}

#creamos las variables relacionadas con el ENDPOINT
url = "https://moviesdatabase.p.rapidapi.com/titles"

API_key = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'
headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}

titleType_list = ['movie', 'short']
genere_list = ['Drama', 'Comedy', 'Action']

querystring = {"titleType":"movie","startYear":"2010","endYear":"2024","genre":"Drama"}


### Vamos a investigar primero en una película del género Drama y tipo movie

In [30]:
#realizamos la llamada
response = requests.get(url, headers=headers, params=querystring)
response.status_code

200

In [31]:
#convertimos la respuesta a json formato
js_response = response.json()
js_response.keys()

dict_keys(['page', 'next', 'entries', 'results'])

In [32]:
#inspeccionamos cada una de las keys()

In [33]:
#Nos da informacion no necesaria
js_response['page']

1

In [34]:
# Nos da el link de la proxima pagina
js_response['next']


'/titles?titleType=short&endYear=2024&startYear=2010&genre=Comedy&page=2'

In [35]:
#Nos da informacion no necesaria
js_response['entries']

10

In [36]:
#guardamos la información relevante en una variable
info = js_response['results']
info

[{'_id': '61e5a8c0e1a8ddbd964aee9b',
  'id': 'tt15072356',
  'primaryImage': None,
  'titleType': {'displayableProperty': {'value': {'plainText': 'Short',
     '__typename': 'Markdown'},
    '__typename': 'DisplayableTitleTypeProperty'},
   'text': 'Short',
   'id': 'short',
   'isSeries': False,
   'isEpisode': False,
   'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}],
   'canHaveEpisodes': False,
   '__typename': 'TitleType'},
  'titleText': {'text': 'The Infiltration Ordeal', '__typename': 'TitleText'},
  'originalTitleText': {'text': 'The Infiltration Ordeal',
   '__typename': 'TitleText'},
  'releaseYear': {'year': 2024, 'endYear': None, '__typename': 'YearRange'},
  'releaseDate': {'day': 1,
   'month': 8,
   'year': 2024,
   '__typename': 'ReleaseDate'}},
 {'_id': '61e5cc95cffb8f2faa5376a7',
  'id': 'tt15869926',
  'primaryImage': None,
  'titleType': {'displayableProperty': {'value': {'plainText': 'Short',
     '__typename': 'Markdown'},
    '__typename': 

In [37]:
#comprobamos el tamaño de la información 
len(info)

10

### 10 listas = 10 peliculas

Cada lista es un diccionario = diccionario de cada pelicula

Keys de cada diccionario:
['_id', 'id', 'primaryImage', 'titleType', 'titleText', 'originalTitleText', 'releaseYear', 'releaseDate']



In [38]:
#Inspeccionamos una lista:
info[0].keys()

dict_keys(['_id', 'id', 'primaryImage', 'titleType', 'titleText', 'originalTitleText', 'releaseYear', 'releaseDate'])

In [39]:
#inspeccionamos cada una de las keys()

In [40]:
# Nos da info no necesaria
info[0]['_id']

'61e5a8c0e1a8ddbd964aee9b'

In [41]:
# Nos da info no necesaria
info[0]['primaryImage']

In [42]:
# Nos da info no necesaria
info[0]['originalTitleText']

{'text': 'The Infiltration Ordeal', '__typename': 'TitleText'}

In [43]:
#Nos da la información sobre el mes de esterno
# Hay que hacer un try/except ya que hay películas que no tienen esta información ya que aún no se han estrenado
month_peli = info[5]['releaseDate']['month'] 
month_peli

In [44]:
# Nos da la información sobre el año de esterno
year_peli = info[0]['releaseYear']['year']
year_peli

2024

In [19]:
#  Nos da la información sobre el nombre de la película
nombre_peli = info[0]['titleText']['text']
nombre_peli

"Mom's Coming"

In [45]:
# Nos da la información sobre el tipo
tipo_peli= info[0]['titleType']['text']
tipo_peli

'Short'

In [46]:
#Nos da la información sobre el id
id_peli = info[0]['id']
id_peli

'tt15072356'

### Vamos a buscar y almacenar todas las paginas 

In [47]:
#creamos dos listas para poder saber que hace el loop while en cada paso por si tenemos errores
lista_respuestas = []
lista_next = []

#iniciamos el loop while para navegar por las distintas páginas
while url.startswith('https'):

    #realizmos la llamada
    response_pelis = requests.get(url, headers=headers, params=querystring)
    lista_respuestas.append(response_pelis.status_code)
    
    #convertimos los resulatos a formato json
    js_response_pelis = response_pelis.json()
   
    #guardamos la inforación de la API dónde tenemos la dirección de la siguiente página
    next = js_response_pelis['next']
    lista_next.append(next)
   
    #guardamos la parte donde tenemos la infomación en una variable
    info_pelis = js_response_pelis['results']

    #guardamos toda la información en el diccionario con un for loop    
    for peli in info_pelis:
        
        dic_API_Pelis['Tipo'].append(peli['titleType']['text'])
        dic_API_Pelis['Nombre'].append(peli['titleText']['text'])
        dic_API_Pelis['Id'].append(peli['id'])
        dic_API_Pelis['Año_estreno'].append(peli['releaseYear']['year'])

        #Hacemos un try/expect por si falta alguna información             
        try:
            dic_API_Pelis['Mes_estreno'].append(peli['releaseDate']['month'])
        except:            
            dic_API_Pelis['Mes_estreno'].append('por estrenar')  
    

    #iniciamos el loop try-except para que cree la variable de la nueva url mientras tengamos una url en ´next´
    try:

        #definimos la base de la nueva url
        url_fixed = "https://moviesdatabase.p.rapidapi.com"
        #definimos la nueva url
        url=url_fixed+next
        #definimos el nuevo querystring
        querystring = {}

    except:
        print('fin')
        break



KeyError: 'next'

In [51]:
respuestas[-5:]

[200, 200, 200, 200, 429]

In [23]:
#Visualizamos nuestro diccionario
dic_API_Pelis

{'Tipo': ['Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  'Movie',
  

In [24]:
# Comprobamos la cantidad de películas/cortos que tenemos
len(dic_API_Pelis['Nombre'])

1147

In [25]:
#Almacenamos la infomacion del diccionario en distintas listas, cada una corresponde a una key 
lista_tipos = dic_API_Pelis['Tipo']
lista_nombre = dic_API_Pelis['Nombre']
lista_anno = dic_API_Pelis['Año_estreno']
lista_mes = dic_API_Pelis['Mes_estreno']
lista_id = dic_API_Pelis['Id']

In [26]:
#Creamos un zip con las listas que hemos creado para que nos guarde la infoamción de cada una de las pelis en una tupla
zip_pelis = zip(lista_tipos, lista_nombre, lista_anno, lista_mes, lista_id)
lista_tuplas_pelis = list(zip_pelis)
lista_tuplas_pelis

[('Movie', "Mom's Coming", 2024, 11, 'tt10579986'),
 ('Movie', 'Love Song and Power', 2024, 1, 'tt10597228'),
 ('Movie', 'Spaceman', 2024, 3, 'tt11097384'),
 ('Movie', 'If - Låtsaskompisar', 2024, 5, 'tt11152168'),
 ('Movie', 'The Accompanist Awakening', 2024, 12, 'tt11237950'),
 ('Movie', 'Manje Bistre 3', 2024, 7, 'tt11822244'),
 ('Movie', 'Eternal Oath', 2024, 7, 'tt12053936'),
 ('Movie', 'The Passion of the Gods', 2024, 6, 'tt12193500'),
 ('Movie', 'Escaping Tomorrow', 2024, None, 'tt12255500'),
 ('Movie', 'After dark', 2024, 12, 'tt12767592'),
 ('Movie', 'Heaven and Hell', 2024, 7, 'tt12861438'),
 ('Movie', 'Broken Glass', 2024, 2, 'tt13036100'),
 ('Movie', 'A Quiet Place: Day One', 2024, 6, 'tt13433802'),
 ('Movie', 'Three Couples Freed', 2024, 1, 'tt13561128'),
 ('Movie', 'Vo Ladki', 2024, 11, 'tt13576680'),
 ('Movie', 'Gina', 2024, 9, 'tt1365464'),
 ('Movie', "We Don't Say Retard Anymore", 2024, 4, 'tt13695406'),
 ('Movie', 'Reckless', 2024, 11, 'tt13965596'),
 ('Movie', 'An Ir

In [27]:
#comprobamos si tenemos todas las pelis que queríamos
len(lista_tuplas_pelis)

1147

In [28]:
# Visualizamos una pelicula al azar
lista_tuplas_pelis[499]

('Movie', 'Il Lago Incantato', 2024, 1, 'tt31149981')